In [1]:
import os
import numpy as np
import pandas as pd
import featuretools as ft
import matplotlib.pyplot as plt
import hyperopt
import seaborn as sns
%matplotlib inline
sns.set(style="ticks", color_codes=True)
from scipy import stats
from scipy.cluster.vq import kmeans2, whiten
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score
from sklearn.preprocessing import RobustScaler
from sklearn.impute import KNNImputer
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import train_test_split

train = pd.read_csv("train.csv",parse_dates=["pickup_time","drop_time"])

In [2]:
train = pd.read_csv("train.csv",parse_dates=["pickup_time","drop_time"])
pd.set_option('display.max_columns', None)

#drop outlier
train = train[train.drop_lat != 48.132000]

#genrate new time related features
data = [train]
for i in data:
   # i['diff'] = i['drop_time'].dt.day - i['pickup_time'].dt.day
    i['Year'] = i['pickup_time'].dt.year
    i['Month'] = i['pickup_time'].dt.month
    i['Date'] = i['pickup_time'].dt.day
    i['Day of Week'] = i['pickup_time'].dt.dayofweek
    i['Hour'] = i['pickup_time'].dt.hour
    
#train['is_weekend'] = train['pickup_time'].apply(lambda x: 1 if x.date().weekday() in (5, 6) else 0)
#train['is_rush_hour'] = train['Hour'].apply(lambda x: 0 if x in (0, 1,2,3,4,5, 6,7,22,23) else 1)


correctness_map = {'correct': 1, 'incorrect': 0}
train['is_FairCorrect']= train['label'].map(correctness_map)
train = train.drop(['label','pickup_time','drop_time'], axis = 1)

#Calculate trip distance
def haversine_distance(lat1, long1, lat2, long2):
    data = [train]
    for i in data:
        R = 6371  #radius of earth in kilometers
        #R = 3959 #radius of earth in miles
        phi1 = np.radians(i[lat1])
        phi2 = np.radians(i[lat2])
    
        delta_phi = np.radians(i[lat2]-i[lat1])
        delta_lambda = np.radians(i[long2]-i[long1])
    
        #a = sin²((φB - φA)/2) + cos φA . cos φB . sin²((λB - λA)/2)
        a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    
        #c = 2 * atan2( √a, √(1−a) )
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
        #d = R*c
        d = (R * c) #in kilometers
        i['H_Distance'] = d
    return d
haversine_distance('pick_lat', 'pick_lon', 'drop_lat', 'drop_lon')


#K-Means clustering for pickup location 
X=train.loc[:,['tripid','pick_lat','pick_lon']]
kmeans = KMeans(n_clusters = 3, init ='k-means++')
kmeans.fit(X[X.columns[1:3]]) # Compute k-means clustering.
X['Pick_cluster_label'] = kmeans.fit_predict(X[X.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(X[X.columns[1:3]]) # Labels of each point
X = X[['tripid','Pick_cluster_label']]
train=train.merge(X, left_on='tripid', right_on='tripid')

#K-Means clustering for pickup location
X=train.loc[:,['tripid','drop_lat','drop_lon']]
kmeans = KMeans(n_clusters = 3, init ='k-means++')
kmeans.fit(X[X.columns[1:3]]) # Compute k-means clustering.
X['Drop_cluster_label'] = kmeans.fit_predict(X[X.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(X[X.columns[1:3]]) # Labels of each point
X = X[['tripid','Drop_cluster_label']]
train=train.merge(X, left_on='tripid', right_on='tripid')

#compute new ratio features
train['additional_fare + meter_waiting_fare' ] = train['additional_fare'] + train['meter_waiting_fare']
train['meter_waiting + meter_waiting_till_pickup'] = train['meter_waiting'] + train['meter_waiting_till_pickup']
train['duration - meter_waiting - meter_waiting_till_pickup'] = train['duration'] - train['meter_waiting'] - train['meter_waiting_till_pickup'] 
train['H_Distance/fare -(additional_fare+meter_waiting_fare)'] = train['H_Distance'] / ((train['fare'] - train['additional_fare'] - train['meter_waiting_fare'] ))
train['H_Distance/duration - meter_waiting - meter_waiting_till_pickup'] = train['H_Distance'] / ( train['duration'] - train['meter_waiting'] - train['meter_waiting_till_pickup'] )
train['additional_fare/H_Distance'] = train['additional_fare'] / train['H_Distance']
train['additional_fare/duration'] = train['additional_fare'] / train['duration']
train['additional_fare/fare'] = train['additional_fare'] / train['fare']
train['fare -(additional_fare+meter_waiting_fare)/fare'] =  (train['fare'] - train['additional_fare'] - train['meter_waiting_fare'] )/ train['fare']
train['fare/duration'] = train['fare'] / train['duration']
train['fare/additional_fare'] = train['fare'] / train['additional_fare']
train['fare/meter_waiting_fare'] = train['fare'] / train['meter_waiting_fare']
train['meter_waiting/duration'] = train['meter_waiting'] / train['duration']
train['meter_waiting_till_pickup/duration'] = train['meter_waiting_till_pickup'] / train['duration']
train['(meter_waiting+meter_waiting_till_pickup)/duration'] = (train['meter_waiting']+train['meter_waiting_till_pickup'])/ train['duration']
train['additional_fare+meter_waiting_fare/fare'] = (train['additional_fare']+train['meter_waiting_fare'])/train['fare']


x_train = train.iloc[:,train.columns!='is_FairCorrect']
y = train['is_FairCorrect'].values
x_train =x_train.drop(['tripid'], axis = 1)
x_train, X_test, y, y_test = train_test_split(x_train, y, test_size=0.3, random_state=22)


In [4]:
from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

#without hyperparameter tuning
X_train=x_train
y_train=y

print("########################################################------------XGBoosting model")
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)
print('sklearn Macro-F1-Score:', f1_score(y_test, y_pred, average='macro'))

print("########################################################------------Light GBM model")
import lightgbm as lgb

d_train = lgb.Dataset(x_train, label=y)
params = {}
clf = lgb.train(params, d_train, 1000)
y_pred=clf.predict(X_test)
x=len(y_pred)
for i in range(0,x):
    if y_pred[i]>=.62:       
       y_pred[i]=int(1)
    else:  
       y_pred[i]=int(0)
    
report = classification_report(y_test, y_pred)
print(report)
print('sklearn Macro-F1-Score lightgbm:', f1_score(y_test, y_pred, average='macro'))

print("########################################################------------CatBoost model")

from catboost import CatBoostClassifier
clf = CatBoostClassifier(logging_level='Silent')

clf.fit(x_train,y)
y_pred = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)
print('sklearn Macro-F1-Score:', f1_score(y_test, y_pred, average='macro'))


########################################################------------XGBoosting model
              precision    recall  f1-score   support

           0       0.85      0.66      0.75       491
           1       0.97      0.99      0.98      4679

    accuracy                           0.96      5170
   macro avg       0.91      0.83      0.86      5170
weighted avg       0.95      0.96      0.95      5170

sklearn Macro-F1-Score: 0.8607911855709003
########################################################------------Light GBM model
              precision    recall  f1-score   support

           0       0.80      0.72      0.76       491
           1       0.97      0.98      0.98      4679

    accuracy                           0.96      5170
   macro avg       0.88      0.85      0.87      5170
weighted avg       0.95      0.96      0.95      5170

sklearn Macro-F1-Score lightgbm: 0.865565056494701
########################################################------------CatBoost model


In [15]:
catboost_params={'bagging_temperature': 71.80681746568663, 'colsample_bylevel': 0.966503669641431, 'learning_rate': 0.10914978299934813, 'max_depth': 8.0, 'random_strength': 56.971535926357895, 'scale_pos_weight': 0.1}



clf = CatBoostClassifier(logging_level='Silent',**catboost_params)

clf.fit(x_train,y,early_stopping_rounds=10)
y_pred4 = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)
print('sklearn Macro-F1-Score:', f1_score(y_test, y_pred, average='macro'))

              precision    recall  f1-score   support

           0       0.80      0.74      0.76       491
           1       0.97      0.98      0.98      4679

    accuracy                           0.96      5170
   macro avg       0.88      0.86      0.87      5170
weighted avg       0.96      0.96      0.96      5170

sklearn Macro-F1-Score: 0.8706017165616815


In [17]:
clf = xgb.XGBClassifier(ccp_alpha=0.0,
criterion='friedman_mse',scale_pos_weight=0.8,min_child_weight=10,gamma=1,colsample_bytree=1.0,early_stopping_rounds=15,

                                                        init=None,
                                                        learning_rate=0.01,
                                                        loss='deviance',
                                                        max_depth=4,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        n_estimators=500,
                                                        subsample=1,
                                                        tol=0.0001,
                                                        validation_fraction=0.1,
                                                        verbose=0,
                                                        warm_start=False)
clf.fit(x_train,y)
y_pred2 = clf.predict(X_test)
print(report)
print('sklearn Macro-F1-Score XGBClassifier2:', f1_score(y_test, y_pred, average='macro'))

              precision    recall  f1-score   support

           0       0.80      0.74      0.76       491
           1       0.97      0.98      0.98      4679

    accuracy                           0.96      5170
   macro avg       0.88      0.86      0.87      5170
weighted avg       0.96      0.96      0.96      5170

sklearn Macro-F1-Score XGBClassifier2: 0.8706017165616815


In [27]:

catboost_params={'l2_leaf_reg': 14.598474686121913, 'learning_rate': 0.03414412531472616, 'max_depth': 9.0,'bagging_temperature': 69.58522316829598, 'colsample_bylevel': 0.9563182212399045,  'random_strength': 72.35482389372854}
clf = CatBoostClassifier(logging_level='Silent',**catboost_params)

clf.fit(x_train,y)
y_pred5 = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)
print('sklearn Macro-F1-Score:', f1_score(y_test, y_pred, average='macro'))

              precision    recall  f1-score   support

           0       0.88      0.64      0.74       491
           1       0.96      0.99      0.98      4679

    accuracy                           0.96      5170
   macro avg       0.92      0.82      0.86      5170
weighted avg       0.96      0.96      0.95      5170

sklearn Macro-F1-Score: 0.8602789313828798


In [16]:
clf = xgb.XGBClassifier(ccp_alpha=0.0,learning_rate= 0.1705,
criterion='friedman_mse',scale_pos_weight=0.33,min_child_weight=0.2881,gamma=4.58,colsample_bytree=0.8799999999999996,subsample=0.79,
                                                        init=None,
                                                        loss='deviance',
                                                        max_depth=7,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        n_estimators=500,
                                                        tol=0.0001,
                                                        validation_fraction=0.1,
                                                        verbose=0,colsample_bylevel=1.0,
                                                        warm_start=False)
clf.fit(x_train, y)
y_pred3 = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

print('sklearn Macro-F1-Score XGBClassifier:', f1_score(y_test, y_pred, average='macro'))

              precision    recall  f1-score   support

           0       0.80      0.74      0.76       491
           1       0.97      0.98      0.98      4679

    accuracy                           0.96      5170
   macro avg       0.88      0.86      0.87      5170
weighted avg       0.96      0.96      0.96      5170

sklearn Macro-F1-Score XGBClassifier: 0.8706017165616815


In [18]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier(logging_level='Silent',scale_pos_weight=0.2)

clf.fit(x_train,y,early_stopping_rounds=15)
y_pred1 = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)
print('sklearn Macro-F1-Score:', f1_score(y_test, y_pred, average='macro'))

              precision    recall  f1-score   support

           0       0.80      0.74      0.76       491
           1       0.97      0.98      0.98      4679

    accuracy                           0.96      5170
   macro avg       0.88      0.86      0.87      5170
weighted avg       0.96      0.96      0.96      5170

sklearn Macro-F1-Score: 0.8706017165616815


In [26]:
#after Max voting ensemble
import statistics
final_pred = np.array([])
for i in range(0,len(X_test)):
    final_pred = np.append(final_pred,statistics.mode([y_pred1[i], y_pred2[i],y_pred3[i], y_pred4[i],y_pred5[i]]))
    
print('sklearn Macro-F1-Score:', f1_score(y_test, final_pred, average='macro'))

sklearn Macro-F1-Score: 0.8707744345523399
